In [1]:
import pandas as pd
import os

In [2]:
df=pd.read_excel('spamhamV2.xlsx', sep=',')

In [3]:
df['Email']=df.Email.str.replace(':','')
df['Email']=df.Email.str.replace('"','')
df['Email']=df.Email.str.replace("'","")
df['Email']=df.Email.str.replace('(','')
df['Email']=df.Email.str.replace(')','')
df['Email']=df.Email.str.replace('\n','')
df['Email']=df.Email.str.replace(',','')
df['Email']=df.Email.str.replace('.','')
df['Email']=df.Email.str.lower()


In [4]:
df['Email']=df['Email'].str.split(pat=None,n=-1,expand=False)


In [5]:
dfTreinamento=df.sample(n=None, frac=0.75, replace=False, weights=None, random_state=None, axis=None)
dfTeste=df.sample(n=None, frac=0.25, replace=False, weights=None, random_state=None, axis=None)

In [6]:
dfTreinamento.to_csv('var_Treinamento.csv')
dfTeste.to_csv('var_Teste.csv')

In [7]:
#Calculando a probabilidade de ser Spam dentro do dataframe de Treinamento
Spam=dfTreinamento[dfTreinamento.Class=='spam']
Pspam=len(Spam)/len(dfTreinamento['Class'])
Pspam

0.13376405838717398

In [8]:
#Calculando a probabilidade de ser Ham dentro do dataframe de Treinamento
Ham=dfTreinamento[dfTreinamento.Class=='ham']
Pham=1-Pspam
Pham

0.866235941612826

In [9]:
dfTotal=dfTreinamento['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie = pd.Series(dfTotal)
palavras = serie.value_counts()
#j=len(dfTotal)

In [11]:
#Calculando a probabilidade de uma palavra acontecer se a mensagem na base treinamento é considerada SPAM
dfTreiSpam=dfTreinamento[dfTreinamento.Class=='spam']
dfTreiSpam=dfTreiSpam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
SpamSeries=pd.Series(dfTreiSpam)

PS=SpamSeries.value_counts()

#j=len(Spam)+len(Ham)
PalavrasSpam=(PS+1)/(len(SpamSeries)+len(palavras))
#PalavrasSpam

In [12]:
#Calculando a probabilidade de uma palavra acontecer se a mensagem na base treinamento é considerada Ham
dfTreiHam=dfTreinamento[dfTreinamento.Class=='ham']
dfTreiHam=dfTreiHam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
HamSeries=pd.Series(dfTreiHam)

PH=HamSeries.value_counts()

PalavrasHam=(PH+1)/(len(HamSeries)+len(palavras))
#PalavrasHam

In [14]:
def Bayes(email):   
    var_s=Pspam
    var_h=Pham
    
    for i in email:
        if i in (PalavrasHam):
            var_h*=PalavrasHam[i]
        else:
            var_h*=1/(len(palavras))
    
    for i in email:
        if i in PalavrasSpam:
            var_s*=PalavrasSpam[i]
        else:
            var_s*=1/(len(palavras))
    
    if var_s > var_h:
        return("spam")
    else:
        return("ham")

In [18]:
def Tester(Teste):
    frase = Teste.Email
    return Bayes(frase)

T = dfTeste.apply(Tester,axis=1)
dfTeste["Resultados"] = T
dfTeste

,Email,Class,Resultados
2256,"[here, got, lots, of, hair, dresser, fr, china]",ham,ham
1245,"[hello, which, the, site, to, download, songs,...",ham,ham
5457,"[arun, can, u, transfr, me, d, amt]",ham,ham
318,"[december, only!, had, your, mobile, 11mths+?,...",spam,spam
4940,"[omg, you, can, make, a, wedding, chapel, in, ...",ham,ham
4979,"[so, what, u, doing, today?]",ham,ham
4797,"[just, come, home, i, dont, want, u, to, be, m...",ham,ham
684,"[i, wanted, to, ask, ì_, to, wait, 4, me, to, ...",ham,ham
3122,"[he, telling, not, to, tell, any, one, if, so,...",ham,ham
700,"[how, much, r, ì_, willing, to, pay?]",ham,ham


In [20]:
pd.crosstab(dfTeste.Class,dfTeste.Resultados)

Resultados,ham,spam
Class,,
ham,1161,35
spam,21,176
